# Requirements

In [1]:
import qpsolvers
from os.path import exists
import pickle

import sn_bayes
from sn_bayes import utils 
from sn_bayes.utils import predict_proba_adjusted
from sn_bayes import longevity_bayes
from sn_bayes.utils import complexity_check
from sn_bayes.utils import get_var_positions
from sn_bayes.utils import get_var_val_positions
from sn_bayes.utils import make_tree
from sn_bayes.utils import bayesInitialize
from sn_bayes.utils import query
from sn_bayes.utils import internal_query
from sn_bayes.utils import internal_query
from sn_bayes.utils import explain_why_bad
from sn_bayes.utils import explain_why_good
from sn_bayes.utils import create_query
from sn_bayes.utils import readable
from sn_bayes.utils import var_deps
from sn_bayes.utils import get_internal_var_val_positions
from sn_bayes.utils import bayesInitialize
from sn_bayes.utils import any_of
from sn_bayes.utils import all_of
from sn_bayes.utils import avg
from sn_bayes.utils import if_then_else
from sn_bayes.utils import bayesInitialize
from sn_bayes.utils import addCpt
from sn_bayes.utils import dependency
from sn_bayes.utils import dependency_direct
from sn_bayes.utils import non_cpt_descriptions

import sn_service.service_spec.bayesian_pb2
from sn_service.service_spec.bayesian_pb2 import Query
import sn_service.service_spec.bayesian_pb2_grpc as grpc_bayes_grpc
import sn_service.service_spec.bayesian_pb2
from sn_service.service_spec.bayesian_pb2 import BayesianNetworkQuery
from sn_service.service_spec.bayesian_pb2 import QueryId
from sn_service.service_spec.bayesian_pb2 import Id


import sn_service.service_spec.bayesian_pb2
from sn_service.service_spec.bayesian_pb2 import BayesianNetwork

import grpc
import pandas as pd
import networkx as nx
import time
import re

import matplotlib.pyplot as plt

In [2]:
bayesianNetwork = BayesianNetwork()
cpt = {}
outstr = ''

discreteDistribution = bayesianNetwork.discreteDistributions.add()
discreteDistribution.name = "bmi"
variable = discreteDistribution.variables.add()
variable.name = "bmi_over_40_high_risk"
variable.probability = 0.08
variable = discreteDistribution.variables.add()
variable.name = "bmi_35_to_39_moderate_risk"
variable.probability = 0.10
variable = discreteDistribution.variables.add()
variable.name = "bmi_30_to_34_low_risk"
variable.probability = 0.20
variable = discreteDistribution.variables.add()
variable.name = "bmi_25_to_29_overweight"
variable.probability = 0.30
variable = discreteDistribution.variables.add()
variable.name = "bmi_under_25_healthy"
variable.probability = 0.32

discreteDistribution = bayesianNetwork.discreteDistributions.add()
discreteDistribution.name = "dietary_energy"
variable = discreteDistribution.variables.add()
variable.name = "dietary_energy_above_3939.00"
variable.probability = 0.05
variable = discreteDistribution.variables.add()
variable.name = "dietary_energy_above_2612.00_to_3939.00_and_below"
variable.probability = 0.20
variable = discreteDistribution.variables.add()
variable.name = "dietary_energy_above_1930.00_to_2612.00_and_below"
variable.probability = 0.25
variable = discreteDistribution.variables.add()
variable.name = "dietary_energy_above_1399.00_to_1930.00_and_below"
variable.probability = 0.25
variable = discreteDistribution.variables.add()
variable.name = "dietary_energy_1399.00_and_below"
variable.probability = 0.25

discreteDistribution = bayesianNetwork.discreteDistributions.add()
discreteDistribution.name = "shortness_of_breath_exertion"
variable = discreteDistribution.variables.add()
variable.name = "shortness_of_breath_exertion_yes"
variable.probability = 0.38
variable = discreteDistribution.variables.add()
variable.name = "shortness_of_breath_exertion_no"
variable.probability = 0.62

discreteDistribution = bayesianNetwork.discreteDistributions.add()
discreteDistribution.name = "daily_walk_or_bicycle_travel"
variable = discreteDistribution.variables.add()
variable.name = "daily_walk_or_bicycle_travel_less_than_30_minutes"
variable.probability = 0.34
variable = discreteDistribution.variables.add()
variable.name = "daily_walk_or_bicycle_travel_30_to_59_minutes"
variable.probability = 0.30
variable = discreteDistribution.variables.add()
variable.name = "daily_walk_or_bicycle_travel_60_minutes_or_more"
variable.probability = 0.36

discreteDistribution = bayesianNetwork.discreteDistributions.add()
discreteDistribution.name = "daily_time_sitting"
variable = discreteDistribution.variables.add()
variable.name = "daily_time_sitting_12_or_more_hours"
variable.probability = 0.08
variable = discreteDistribution.variables.add()
variable.name = "daily_time_sitting_over_6_but_less_than_12_hours"
variable.probability = 0.36
variable = discreteDistribution.variables.add()
variable.name = "daily_time_sitting_over_2_but_less_than_6_hours"
variable.probability = 0.49
variable = discreteDistribution.variables.add()
variable.name = "daily_time_sitting_less_than_2_hours"
variable.probability = 0.06

discreteDistribution = bayesianNetwork.discreteDistributions.add()
discreteDistribution.name = "physical_work"
variable = discreteDistribution.variables.add()
variable.name = "physical_work_no"
variable.probability = 0.76
variable = discreteDistribution.variables.add()
variable.name = "physical_work_yes"
variable.probability = 0.24

discreteDistribution = bayesianNetwork.discreteDistributions.add()
discreteDistribution.name = "ten_minutes_daily_workout"
variable = discreteDistribution.variables.add()
variable.name = "ten_minutes_daily_workout_no"
variable.probability = 0.60
variable = discreteDistribution.variables.add()
variable.name = "ten_minutes_daily_workout_yes"
variable.probability = 0.40

discreteDistribution = bayesianNetwork.discreteDistributions.add()
discreteDistribution.name = "steps_anomaly"
variable = discreteDistribution.variables.add()
variable.name = "steps_anomaly"
variable.probability = 0.05
variable = discreteDistribution.variables.add()
variable.name = "no_steps_anomaly"
variable.probability = 0.95

discreteDistribution = bayesianNetwork.discreteDistributions.add()
discreteDistribution.name = "workout_anomaly"
variable = discreteDistribution.variables.add()
variable.name = "workout_anomaly"
variable.probability = 0.05
variable = discreteDistribution.variables.add()
variable.name = "no_workout_anomaly"
variable.probability = 0.95

discreteDistribution = bayesianNetwork.discreteDistributions.add()
discreteDistribution.name = "walking_speed_anomaly"
variable = discreteDistribution.variables.add()
variable.name = "walking_speed_anomaly"
variable.probability = 0.05
variable = discreteDistribution.variables.add()
variable.name = "no_walking_speed_anomaly"
variable.probability = 0.95

In [3]:
cpt["lack_of_workout_reported"] = all_of(bayesianNetwork,cpt,
                {
                    "daily_walk_or_bicycle_travel":"daily_walk_or_bicycle_travel_less_than_30_minutes",
                    "physical_work":"physical_work_no",
                    'ten_minutes_daily_workout':"ten_minutes_daily_workout_no"
                        },
                ["lack_of_workout_reported","no_lack_of_workout_reported"]
                )
cpt

{'lack_of_workout_reported': ([['daily_walk_or_bicycle_travel_less_than_30_minutes',
    'physical_work_no',
    'ten_minutes_daily_workout_no',
    'lack_of_workout_reported',
    1.0],
   ['daily_walk_or_bicycle_travel_less_than_30_minutes',
    'physical_work_no',
    'ten_minutes_daily_workout_no',
    'no_lack_of_workout_reported',
    0.0],
   ['daily_walk_or_bicycle_travel_less_than_30_minutes',
    'physical_work_no',
    'ten_minutes_daily_workout_yes',
    'lack_of_workout_reported',
    0.0],
   ['daily_walk_or_bicycle_travel_less_than_30_minutes',
    'physical_work_no',
    'ten_minutes_daily_workout_yes',
    'no_lack_of_workout_reported',
    1.0],
   ['daily_walk_or_bicycle_travel_less_than_30_minutes',
    'physical_work_yes',
    'ten_minutes_daily_workout_no',
    'lack_of_workout_reported',
    0.0],
   ['daily_walk_or_bicycle_travel_less_than_30_minutes',
    'physical_work_yes',
    'ten_minutes_daily_workout_no',
    'no_lack_of_workout_reported',
    1.0],
   ['

In [4]:
cpt["lack_of_exercise_reported"] = any_of(bayesianNetwork,cpt,
                {
                    "shortness_of_breath_exertion":"shortness_of_breath_exertion_yes",
                    "daily_time_sitting":"daily_time_sitting_12_or_more_hours",
                    "lack_of_workout_reported":"lack_of_workout_reported"
                        },
                ["lack_of_exercise_reported","no_lack_of_exercise_reported"]
                )
cpt

{'lack_of_workout_reported': ([['daily_walk_or_bicycle_travel_less_than_30_minutes',
    'physical_work_no',
    'ten_minutes_daily_workout_no',
    'lack_of_workout_reported',
    1.0],
   ['daily_walk_or_bicycle_travel_less_than_30_minutes',
    'physical_work_no',
    'ten_minutes_daily_workout_no',
    'no_lack_of_workout_reported',
    0.0],
   ['daily_walk_or_bicycle_travel_less_than_30_minutes',
    'physical_work_no',
    'ten_minutes_daily_workout_yes',
    'lack_of_workout_reported',
    0.0],
   ['daily_walk_or_bicycle_travel_less_than_30_minutes',
    'physical_work_no',
    'ten_minutes_daily_workout_yes',
    'no_lack_of_workout_reported',
    1.0],
   ['daily_walk_or_bicycle_travel_less_than_30_minutes',
    'physical_work_yes',
    'ten_minutes_daily_workout_no',
    'lack_of_workout_reported',
    0.0],
   ['daily_walk_or_bicycle_travel_less_than_30_minutes',
    'physical_work_yes',
    'ten_minutes_daily_workout_no',
    'no_lack_of_workout_reported',
    1.0],
   ['

In [5]:
cpt["lack_of_exercise_signal"] = avg(bayesianNetwork,cpt,
                [
         "workout_anomaly",
         "walking_speed_anomaly",
         "steps_anomaly"
         ],
         ["lack_of_exercise_signal","no_lack_of_exercise_signal"]
                 
         )

In [6]:
cpt_rows,keylist,outvars,description = dependency_direct(bayesianNetwork, cpt, invars, outvars, priors = None,adjust=False)

NameError: name 'invars' is not defined

In [7]:
description

NameError: name 'description' is not defined

In [ ]:
outvars

In [ ]:
keylist

In [ ]:
cpt_rows

In [ ]:
var_positions= get_var_positions(bayesianNetwork0)
var_positions

In [ ]:
df = make_tree(bayesianNetwork0)
G = nx.DiGraph()
regex = re.compile(r'^([a-z_]*)\(.*')
for index, row in df.iterrows():
    for col in df.columns:
        if row[col] is not None:
            node = row[col][:-1] if col == "level0" else row[col]
            G.add_node(node)           
for node1 in G.nodes:
    for node2 in G.nodes:
        match = regex.match(node1)
        if (node1+',' in node2) or (match is not None and match.group(1)+"," in node2): 
            G.add_edge(node1,node2)
            if match is not None and match.group(1)+"," in node2:
                print (node1)
                print(node2)
                
plt.figure(3,figsize=(6,6)) 
nx.draw_circular(G, with_labels=True)

In [ ]:
var_positions = get_var_positions(bayesianNetwork0)
var_positions

In [ ]:
var_val_positions = get_var_val_positions(bayesianNetwork0)
var_val_positions

In [ ]:
internal_var_val_positions = get_internal_var_val_positions(bayesianNetwork0)
internal_var_val_positions

In [ ]:
cpt["lack_of_exercise_reported"] = any_of(bayesianNetwork0,cpt,
                {
                    "shortness_of_breath_exertion":"shortness_of_breath_exertion_yes",
                    "daily_time_sitting":"daily_time_sitting_12_or_more_hours",
                    "lack_of_workout_reported":"lack_of_workout_reported"
                        },
                ["lack_of_exercise_reported","no_lack_of_exercise_reported"]

                )

In [ ]:
# We create a spreadsheet of the above for reference
rows_list = []
outname = "varvals.csv"
for var, valdict in var_val_positions.items():
    rowdict = {} 
    rows_list.append(rowdict)
    varstr= var +"("
    for val, pos in valdict.items():
        varstr += val
        varstr+= ","
            
    varstr=varstr[:-1]+")"
    rowdict["variable"] = varstr        
#
df = pd.DataFrame(rows_list)

In [ ]:
df